Para iniciar os experimentos, precisamos construir a nossa base de dados. Vamos utilizar como exemplo a base CALTECH-101 (http://www.vision.caltech.edu/Image_Datasets/Caltech101/Caltech101.html).
Essa base de dados contém 101 objetos diferentes com um conjunto de 40 a 800 imagens por categoria. A categoria que escolhemos para trabalharmos no nosso detector foi a `Faces`, que contém imagens de rostos de pessoas.
Sabendo que a base do detector de objetos é um classificador, precisamos de exemplos do que são esses objetos e exemplos do que não consiste no objeto de interesse. Para isso vamos usar a 13 Natural Scene Category (http://vision.stanford.edu/resources_links.html#datasets).

Vamos primeiro especificar os caminhos para os nossos dados.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

conf = {
    'faces_dataset': '/gdrive/MyDrive/Mestrado/VisaoComputacional/Faces',
    'image_data': '/gdrive/MyDrive/Mestrado/VisaoComputacional/Faces_Annotations',
    'image_distractions': '/gdrive/MyDrive/Mestrado/VisaoComputacional/sceneclass13'
}

Vamos analisar o tamanho médio dos objetos e seu *aspect ratio* médio.

In [ ]:
import numpy as np
import glob
from scipy import io
import cv2
from google.colab.patches import cv2_imshow

widths = []
heights = []

for p in glob.glob(conf["image_data"] + "/*.mat"):
	(y, h, x, w) = io.loadmat(p)["box_coord"][0]
	widths.append(w - x)
	heights.append(h - y)

(avgWidth, avgHeight) = (np.mean(widths), np.mean(heights))
print("[INFO] largura média: {:.2f}".format(avgWidth))
print("[INFO] altura média: {:.2f}".format(avgHeight))
print("[INFO] aspect ratio: {:.2f}".format(avgWidth / avgHeight))

[INFO] largura média: 204.24
[INFO] altura média: 263.34
[INFO] aspect ratio: 0.78


Vamos agora construir uma função que nos ajudará a extrair a região de interesse da imagem que fica dentro do *bounding box*.

In [ ]:
def roi_extract(image, bb, padding=10, dstSize=(42, 42)):
	# com base no bounding box extrai o ROI da imagem

	(y, h, x, w) = bb
	(x, y) = (max(x - padding, 0), max(y - padding, 0))
	roi = image[y:h + padding, x:w + padding]
	# redimensiona a região de interesse para um tamanho desejado, se for o caso.
	roi = cv2.resize(roi, dstSize, interpolation=cv2.INTER_AREA)
	# retorna somente a região de interesse da imagem.
	return roi

Uma das técnicas que será utilizada para detectar os objetos será por meio das pirâmide de imagens.

Para isso vamos implementar duas funções para contruir essa pirâmide.

In [ ]:
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
	
  # inicializa as dimensões da imagem e obtém o tamanho da imagem original.
	dim = None
	(h, w) = image.shape[:2]
	# se ambas a altura e largura forem None, retorna a imagem original.  
	if width is None and height is None:
		return image
	
    # calcula o AR com base na altura.
	if width is None:		
		r = height / float(h)
		dim = (int(w * r), height)
	# caso contrário, calcula o AR com base na largura.
	else:
		r = width / float(w)
		dim = (width, int(h * r))
	
  # redimensiona a imagem.
	resized = cv2.resize(image, dim, interpolation=inter)
	
  # retorna a imagem redimensionada.
	return resized

In [ ]:
def pyramid(image, scale=2.5, minSize=(40, 40)):
	# retorna a imagem original
	yield image
	
  # continua o loop pela pirâmide
	while True:
		# computa as novas dimensões da imagem e a redimensiona    
		w = int(image.shape[1] / scale)
		image = resize(image, width=w)
		
    # verifica se o tamanho da imagem redimensinada atinge é menor que o tamanho
    # mínimo fornecido. 
		if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
      # para a construção da pirâmide
			break
		
    # retorna a próxima imagem da pirâmide
		yield image

Para detectar um objeto, precisamos buscar dentro da imagem por esse objeto.

Para isso vamos implementar um esquema de janelas deslizantes.

In [ ]:
def sliding_window(image, stepSize, windowSize):
	# move a janela através da imagem
	for y in range(0, image.shape[0], stepSize):
		for x in range(0, image.shape[1], stepSize):
			# retorna a porção da imagem referente a janela corrente
			yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

Para extrair as características de cada recorte feito pelas janelas deslizantes, precisamos, primeiro, estruturar os dados de uma maneira que possam ser facilmente acessados e armazenados.

Vamos criar duas funções iniciais para ajudar nisso.

In [ ]:
import numpy as np
import h5py

def dump_dataset(data, labels, path, datasetName, writeMethod="w"):
  # abre banco de dados, cria um dataset e escreve os dados e rótulos.  
  db = h5py.File(path, writeMethod)
  dataset = db.create_dataset(datasetName, (len(data), len(data[0]) + 1), dtype="float")
  dataset[0:len(data)] = np.c_[labels, data]
  db.close()
 
def load_dataset(path, datasetName):
  # abre o banco de dados, coleta os dados e rótulos.
  db = h5py.File(path, "r")
  (labels, data) = (db[datasetName][:, 0], db[datasetName][:, 1:])
  db.close()
  # retorna a tupla de dados e rótulos
  return (data, labels)

Agora vamos juntar tudo para extrair as características das imagens.

In [ ]:
from imutils import paths
import progressbar
import skimage
from skimage import feature
import random

# parâmetro referente ao percentual de utilização da base para treinamento
conf['percent_gt_images'] = 0.5
# tamanho do padding (quantidade de pixels) a ser utilizado no corte da imagem
# é sempre bom deixar uns pixels a mais nas bordas da imagem
conf['offset'] = 5
# tamanho da janela deslizante
conf['window_dim'] = [100, 110]
# número de orientações do descritor HOG
conf['orientations'] = 9
# pixels por célula do HOG
conf['pixels_per_cell'] = [8, 8]
# células por bloco do HOG
conf['cells_per_block'] = [4, 4]

data = []
labels = []

# obtém um conjunto de referência para o treinamento (ground-truth) e separa uma
# parte para treinamento.
trnPaths = list(paths.list_images(conf["faces_dataset"]))
trnPaths = random.sample(trnPaths, int(len(trnPaths) * conf["percent_gt_images"]))
print("[INFO] describing training ROIs...")

In [ ]:
# cria um elemento de progress bar só para acompanhar a evolução do processo.
widgets = ["Extraindo: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=len(trnPaths), widgets=widgets).start()
# loop nos caminhos de arquivos de treinamento
for (i, trnPath) in enumerate(trnPaths):
  image = cv2.imread(trnPath)
  # converte para escala de cinza
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # define um ID para a imagem baseada no nome dela.
  imageID = trnPath[trnPath.rfind("_") + 1:].replace(".jpg", "")
  # carrega a anotação (bounding box) da imagem
  p = "{}/annotation_{}.mat".format(conf["image_data"], imageID)
  bb = io.loadmat(p)["box_coord"][0]
  # extrai o ROI da imagem (corta só o objeto de interesse) usando a função definida
  # antes.
  roi = roi_extract(image, bb, padding=conf["offset"], dstSize=tuple(conf["window_dim"]))

  # realiza o flip na imagem para aumentar a quantidade de dados disponíveis.

  rois = (roi, cv2.flip(roi, 1))

  # loop em cada ROI
  for roi in rois:
    # extrai as características do ROI e atualiza a lista de features e rótulos
    features = feature.hog(roi, conf['orientations'], pixels_per_cell=conf['pixels_per_cell'],
        cells_per_block=conf['cells_per_block'], transform_sqrt=True)
    data.append(features)
    labels.append(1)
  # atualiza a barra de progresso
  pbar.update(i)
pbar.finish()

In [ ]:
image = cv2.imread(list(paths.list_images(conf["faces_dataset"]))[0])
cv2_imshow(image)
cv2_imshow(cv2.flip(image,1))

Uma vez tratado todo o conjunto de exemplos positivos da nossa base de dados, precisamos extrair as características das imagens negativas.

In [ ]:
from sklearn.feature_extraction.image import extract_patches_2d

# define o número de imagens negativas a serem utilizadas
conf['num_distraction_images'] = 500
# define o número de pedaços da imagem negativa que deve ser extraído
conf['num_distractions_per_image'] = 10

# pega as imagens negativas
dstPaths = list(paths.list_images(conf['image_distractions']))
widgets = ["Extraindo: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=conf["num_distraction_images"], widgets=widgets).start()
# loop em uma quantidade de imagens negativas
for i in np.arange(0, conf['num_distraction_images']):
  # aleatoriamente seleciona uma imagem do conjunto negativo
  image = cv2.imread(random.choice(dstPaths))
  # converte para escala de cinza
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # extrai pedaços aleatórios da imagem
  patches = extract_patches_2d(image, tuple(conf["window_dim"]),
    max_patches=conf["num_distractions_per_image"])
  # loop em cada pedaço
  for patch in patches:
    # extrai as características do pedaço e atualiza a lista de dados e rótulos.
    features = feature.hog(patch, conf['orientations'], pixels_per_cell=conf['pixels_per_cell'],
        cells_per_block=conf['cells_per_block'], transform_sqrt=True)
    data.append(features)
    labels.append(-1)
  # atualiza a barra de progresso
  pbar.update(i)
pbar.finish()

A última coisa a fazer é colocar todas as features extraídas no banco de dados construído.

In [ ]:
conf['features_path'] = '/gdrive/MyDrive/Mestrado/VisaoComputacional/outputs/faces_features.hdf5'
print('[INFO] dumping features e rótulos para o arquivo...')
dump_dataset(data, labels, conf['features_path'], 'features')

Depois de conseguir extrair as características do nosso conjunto de dados, incluindo exemplos positivos e negativos, vamos agora para o treinamento do classificador.

Vamos primeiro carregar o nosso conjunto de dados a partir do banco criado anteriormente.

In [ ]:
print("[INFO] carregando dataset...")
(data, labels) = load_dataset(conf['features_path'], 'features')

Aqui vamos executar o processo de treinamento.

In [ ]:
from sklearn.svm import SVC
import pickle

# local para armazenar o classificador após treinado
conf['classifier_path'] = '/gdrive/MyDrive/Mestrado/VisaoComputacional/outputs/classificador.cpickle'

print("[INFO] treinando o classificador...")

model = SVC(kernel="linear", C=0.8, probability=True, random_state=42)
model.fit(data, labels)
# armazenando o classificador no arquivo
print("[INFO] dumping classificador...")
f = open(conf["classifier_path"], "wb")
f.write(pickle.dumps(model))
f.close()

A função abaixo é utilizada para fazer a detecção de um objeto na imagem.

In [ ]:
def detect(image, model, winDim, winStep=4, pyramidScale=2.5, minProb=0.7):
# inicializa a lista de bounding boxes e probabilidades associadas
  boxes = []
  probs = []

  # loop na pirâmide de imagens    
  for layer in pyramid(image, scale=pyramidScale, minSize=winDim):
    # determina a escala atual da pirâmide
    scale = image.shape[0] / float(layer.shape[0])
    # loop da janela deslizante em todas as imagens da pirâmide
    for (x, y, window) in sliding_window(layer, winStep, winDim):
      # pega as dimensões da janela
      (winH, winW) = window.shape[:2]
      # garante que as dimensões da janela se enquadram com os valores fornecidas
      # para a janela deslizante      
      if winH == winDim[1] and winW == winDim[0]:
        # extrai as características HOG da janela atual e classifca para determinar
        # se o objeto de interesse está presente ou não.        
        
        features = feature.hog(window, conf['orientations'], pixels_per_cell=conf['pixels_per_cell'],
        cells_per_block=conf['cells_per_block'], transform_sqrt=True).reshape(1, -1)

        #features = desc.describe(window).reshape(1, -1)
        
        prob = model.predict_proba(features)[0][1]
        # verifica se o classificador encontrou um objeto com probabilidade suficiente        
        if prob > minProb:
          # calcula a coordenada (x,y) do bounding box usando a escala atual da imagem          
          (startX, startY) = (int(scale * x), int(scale * y))
          endX = int(startX + (scale * winW))
          endY = int(startY + (scale * winH))
          # atualiza a lista de bounding boxes e probabilidades          
          boxes.append((startX, startY, endX, endY))
          probs.append(prob)
  # retorna uma tupla de bounding boxes e probabilidades
  return (boxes, probs)

Um problema de fazer a detecção dessa maneira é que múltiplos *bounding box* aparecem no resultado do teste, para evitar isso, vamos utilizar a técnica *non-maxima supression*.

In [ ]:
def non_max_suppression(boxes, probs, overlapThresh):
  # se não existem bbs, retornar uma lista vazia
  if len(boxes) == 0:
    return []
  # converte os valores dos bbs para float já que haverá operações de divisão.
  if boxes.dtype.kind == "i":
    boxes = boxes.astype("float")
  
  # inicializa uma lista para guardar os bbs selecionados
  pick = []
  # pega as coordenadas dos bbs
  x1 = boxes[:, 0]
  y1 = boxes[:, 1]
  x2 = boxes[:, 2]
  y2 = boxes[:, 3]
  # calcula a área dos bbs e os ordena pela probabilidade associada a cada um.
  area = (x2 - x1 + 1) * (y2 - y1 + 1)
  idxs = np.argsort(probs)

  # loop pela lista de indices ordenadas
  while len(idxs) > 0:
    # pega o último indice da lista (maior probabilidade) e adiciona a lista de valores selecionados.
    last = len(idxs) - 1
    i = idxs[last]
    pick.append(i)

    # encontra os maiores valores de (x,y) para ser o início do bounding box e os
    # menores valores (x,y) para ser o fim do bounding box.
    xx1 = np.maximum(x1[i], x1[idxs[:last]])
    yy1 = np.maximum(y1[i], y1[idxs[:last]])
    xx2 = np.minimum(x2[i], x2[idxs[:last]])
    yy2 = np.minimum(y2[i], y2[idxs[:last]])
    # calcula a largura e altura do bounding box
    w = np.maximum(0, xx2 - xx1 + 1)
    h = np.maximum(0, yy2 - yy1 + 1)
    # calcula a taxa de sobreposição
    overlap = (w * h) / area[idxs[:last]]
    # deleta todos os índices que possuem sobreposição maior que limiar
    idxs = np.delete(idxs, np.concatenate(([last],
      np.where(overlap > overlapThresh)[0])))
  # retorna apenas os bounding boxes selecionados
  return boxes[pick].astype("int")

Finalmente, vamos fazer os testes para a deteção de faces nas imagens.

In [ ]:
import imutils

conf['overlap_thresh'] = 0.3

# definindo mais um parâmetro para as janelas deslizantes
conf['window_step'] = 4

# definindo a probabilidade mínima para o classificador considerar que existe um
# objeto na imagem
conf['min_probability'] = 0.7

# define a escala da pirâmide de imagens
conf['pyramid_scale'] = 2.5

model = pickle.loads(open(conf['classifier_path'], 'rb').read())

image = cv2.imread('/gdrive/MyDrive/Mestrado/VisaoComputacional/Faces/image_0141.jpg')
image = imutils.resize(image, width=min(260, image.shape[1]))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# detect objects in the image and apply non-maxima suppression to the bounding boxes
(boxes, probs) = detect(gray, model, conf["window_dim"], winStep=conf["window_step"],
	pyramidScale=conf["pyramid_scale"], minProb=conf["min_probability"])

pick = non_max_suppression(np.array(boxes), probs, conf["overlap_thresh"])
orig = image.copy()

# loop over the original bounding boxes and draw them
for (startX, startY, endX, endY) in boxes:
	cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 0, 255), 2)

# loop over the allowed bounding boxes and draw them
for (startX, startY, endX, endY) in pick:
	cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
# show the output images
cv2_imshow(orig)
cv2_imshow(image)